### 1. EDA를 기반으로 한 데이터 전처리
---

EDA를 통해 분석한 데이터를 통해, 데이터를 전처리 하고 인코딩

모델에 사용한 features, labels

Species         나무 종명   
Light_ISF	    조도 (받는 햇빛의 양)   
Light_Cat	    빛의 세기   
Soil	        토양의 출처     
Sterile	        토양의 살균 여부    
Conspecific	    토양의 타입 (동종에서 채취한 토양, 이종에서 채취한 토양,...)        
Myco	        나무에서 발견되는 균의 종류     
SoilMyco	    토양에서 발견되는 균의 종류     
PlantDate	    식재일      
AMF	            AMF균의 비율        
EMF	            EMF균의 비율        
Phenolics	    페놀 화합물의 함유량        
Lignin	        리그닌의 비율       
NSC	            NSC의 비율      

Time	        사건(사망 혹은 측정 종료)가 발생한 시간     

Alive           사건 종류       
  

전처리 과정 실행        
데이터 드랍 -> 이상치 처리 -> 결측치 처리 -> 날짜값 통일 -> 라벨 데이터 통합 -> 인코딩

### 2. 분류 모델을 이용한 묘목의 생존 여부 예측

분류 모델을 이용하여 2 성장기가 지난 묘목 (약 120일 후)의 묘목의 생존율 예측        
2 성장기가 지나면 생존율은 매우 안정화. 해당 기간까지의 생존율 예측이 매우 중요     


-- 실제 묘목의 생존율 그래프 --

3개의 분류 모델을 사용하여 분류를 수행

- Logistic Regression
- Random Forest
- XGBoost

-- 각 모델 평가 및 결과 분석

### 3. 기존 모델의 한계 및 새로운 모델 기획
- 기존 모델은 관측 종료 시점(해당 데이터에서는 115.5일 후)에서의 생존 여부만 예측 가능
- 실무에서는 다양한 시기의 생존율을 예측해야할 필요가 있고, 또한 적은 데이터만을 가지고 예측하기도 해야함
    - 1. 모델의 일부 특성만을 이용한 모델 구현 (특성 선택 이용)
    - 2. 생존율을 시간에 따라 예측할 수 있는 모델 구현

### 4. 시간에 따른 생존율을 예측하는 모델 구현 및 평가
단순히 생존 여부만을 판단하는 모델이 아니라, 시간에 따른 개체별 위험도와 주기적인 관찰이 필요한 개체 등을 구분하기 위해서는 시간에 따른 생존 곡선을 그릴 수 있는 모델이 필요함

- XGBoost with Cox
- Random Survival Forest

위 두 모델을 기반으로 평가한 결과, RSF가 좋은 성능을 보임

- 위 모델을 기반으로 생존 확률 그래프, 혼동 행렬 등등 출력하고 모델 평가 및 결과 분석

### 5. 모델 평가 방식과 성능 고도화 기법

- 생존율 모델에서 threshold를 이용한 이진 분류 후 정확도 평가
- KFold 를 이용한 모델의 정확도 측정
- GridSearchCV를 이용한 최적의 파라미터 탐색

### 6. 트러블 슈팅
- 생존율 모델의 경우 위험 점수를 기반으로 한 평가 방식으로는 기존 모델과 비교 불가
    - 위험 점수를 기반으로 모델을 이진 분류한 후 정확도를 계산하는 방식으로 기존 모델과 비교

- 생존율 모델의 경우, 특성, 사건 발생 또는 관측 종료 시간, 사건 발생 여부 3개의 데이터를 모델에 전달
- GridSearchCV 등은 모두 X, y 만을 받는 모델을 기반으로 함
    - 인자를 2개 (특성, 시간과 사건) 혹은 3개로 모두 받을 수 있도록 메소드 디자인

- 생존율 예측 -> 생존 : 1, 사망 : 0으로 분류하는것이 자연스럽지만, 때문에 사건 발생인 사망 이 0으로 분류되어 confusion matrix를 기반으로 한 평가 지표에 오류 발생
    - 해당 함수를 새로 정의하여 해결

